In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import time
import pickle
import cv2
%matplotlib inline
import matplotlib.image as mpimg

import tensorflow
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
from sklearn.svm import SVC

In [2]:
def save_model_weights(model,path):
    model.save_weights(path)
    return
def save_model_arch(model,path):
    model.save(path)

In [3]:
vgg19 = tensorflow.keras.applications.VGG19
initial_model=vgg19(weights='imagenet', include_top=True)

In [4]:
initial_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
x = Dense(2, activation='softmax',name='classifier')(initial_model.layers[-2].output)
tune_model = Model(initial_model.input, x)
opt = Adam(learning_rate=0.0001)
tune_model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
tune_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
path = 'E:/ds project/mal_mc_dataset/'
target_size = (224, 224)
batch_size = 10
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
 datagen = ImageDataGenerator()
 return datagen.flow_from_directory(directory=directory,
                                    target_size=target_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=shuffle)
batches = get_batches(path+'train', shuffle=True)
valid_batches = get_batches(path+'validation', shuffle=False)

Found 466 images belonging to 2 classes.
Found 154 images belonging to 2 classes.


In [8]:
reduce = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                      factor=0.1, 
                                                      patience=5, 
                                                      mode='auto')

early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 min_delta=1e-4, 
                                                 patience=10, 
                                                 mode='auto')

out=tune_model.fit(x=batches,
              steps_per_epoch=batches.samples/batches.batch_size, 
              epochs=30, 
              validation_data=valid_batches, 
              validation_steps=valid_batches.samples/valid_batches.batch_size, 
                   callbacks=[reduce, early])

Epoch 1/30
46/46 [==============================] - 732s 16s/step - loss: 1.8389 - accuracy: 0.9056 - val_loss: 4.6224e-06 - val_accuracy: 1.0000
Epoch 2/30
46/46 [==============================] - 648s 14s/step - loss: 0.2639 - accuracy: 0.9528 - val_loss: 7.7409e-09 - val_accuracy: 1.0000
Epoch 3/30
46/46 [==============================] - 647s 14s/step - loss: 1.7907e-09 - accuracy: 1.0000 - val_loss: 7.7409e-10 - val_accuracy: 1.0000
Epoch 4/30
46/46 [==============================] - 622s 13s/step - loss: 1.5349e-09 - accuracy: 1.0000 - val_loss: 7.7409e-10 - val_accuracy: 1.0000
Epoch 5/30
46/46 [==============================] - 614s 13s/step - loss: 1.5349e-09 - accuracy: 1.0000 - val_loss: 7.7409e-10 - val_accuracy: 1.0000
Epoch 6/30
46/46 [==============================] - 631s 14s/step - loss: 1.5349e-09 - accuracy: 1.0000 - val_loss: 7.7409e-10 - val_accuracy: 1.0000
Epoch 7/30
46/46 [==============================] - 619s 13s/step - loss: 1.5349e-09 - accuracy: 1.0000 - va

In [9]:
tune_model.save('mal_ftr_extrn_model_30eph.h5')